In [8]:
import numpy as numpy
import pandas as pd
import scipy as sp
import sklearn
import warnings
import pickle
warnings.filterwarnings('ignore')

In [9]:
from sklearn import linear_model,model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report

In [10]:
#loading data
data_train=pd.read_csv('data/train.csv')
data_val=pd.read_csv('data/test.csv')

In [11]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
data_val.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [13]:
#combining train and validation/test datasets
data_train['test']=0
data_val['test']=1
data_all=data_train.append(data_val)

In [14]:
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
test             0
dtype: int64

In [15]:
print('data columns with null values:\n', data_all.isnull().sum())

data columns with null values:
 PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
test              0
dtype: int64


In [16]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  test         1309 non-null   int64  
dtypes: float64(3), int64(5), object(5)
memory usage: 143.2+ KB


In [17]:
#preprocessing
#complete missing age with median
data_all['Age'].fillna(data_all['Age'].median(), inplace = True)

#complete embarked with mode
data_all['Embarked'].fillna(data_all['Embarked'].mode()[0], inplace = True)

#complete missing fare with median
data_all['Fare'].fillna(data_all['Fare'].median(), inplace = True)

#delete the cabin feature/column and others previously stated to exclude in train dataset
drop_column = ['PassengerId','Cabin', 'Ticket']
data_all.drop(drop_column, axis=1, inplace = True)


In [18]:
data_all

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,test
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,0
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,0
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...,...
413,NaN,3,"Spector, Mr. Woolf",male,28.0,0,0,8.0500,S,1
414,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C,1
415,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,S,1
416,NaN,3,"Ware, Mr. Frederick",male,28.0,0,0,8.0500,S,1


In [19]:
#Discrete variables
data_all['FamilySize'] = data_all['SibSp'] + data_all['Parch'] + 1
data_all

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,test,FamilySize
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0,2
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,0,2
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,0,1
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,0,2
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,3,"Spector, Mr. Woolf",male,28.0,0,0,8.0500,S,1,1
414,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C,1,1
415,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,S,1,1
416,NaN,3,"Ware, Mr. Frederick",male,28.0,0,0,8.0500,S,1,1


In [20]:
data_all['IsAlone'] = 1 #initialize to yes/1 is alone
data_all['IsAlone'].loc[data_all['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1
data_all['IsAlone']

0      0
1      0
2      1
3      0
4      1
      ..
413    1
414    1
415    1
416    1
417    0
Name: IsAlone, Length: 1309, dtype: int64

In [21]:
#quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
data_all['Title'] = data_all['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]



In [22]:
data_all['Title'] 

0          Mr
1         Mrs
2        Miss
3         Mrs
4          Mr
        ...  
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: Title, Length: 1309, dtype: object

In [23]:
#Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
#Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
data_all['FareBin'] = pd.qcut(data_all['Fare'], 4)

#Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
data_all['AgeBin'] = pd.cut(data_all['Age'].astype(int), 5)

#cleanup rare title names
#print(data1['Title'].value_counts())
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (data_all['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
data_all['Title'] = data_all['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)

In [24]:
data_all.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,test,FamilySize,IsAlone,Title,FareBin,AgeBin
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0,2,0,Mr,"(-0.001, 7.896]","(16.0, 32.0]"
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,0,2,0,Mrs,"(31.275, 512.329]","(32.0, 48.0]"
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,0,1,1,Miss,"(7.896, 14.454]","(16.0, 32.0]"
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,0,2,0,Mrs,"(31.275, 512.329]","(32.0, 48.0]"
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0,1,1,Mr,"(7.896, 14.454]","(32.0, 48.0]"


In [25]:
#code categorical data
label = LabelEncoder()
data_all['Sex_Code'] = label.fit_transform(data_all['Sex'])
data_all['Embarked_Code'] = label.fit_transform(data_all['Embarked'])
data_all['Title_Code'] = label.fit_transform(data_all['Title'])
data_all['AgeBin_Code'] = label.fit_transform(data_all['AgeBin'])
data_all['FareBin_Code'] = label.fit_transform(data_all['FareBin'])

In [26]:
Target = ['Survived']
data_all_x_cols = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']

In [34]:
data_all_x_cols

['Sex_Code',
 'Pclass',
 'Embarked_Code',
 'Title_Code',
 'FamilySize',
 'AgeBin_Code',
 'FareBin_Code']

In [27]:
# Only useful id you have test and train data seperate
data_train=data_all[data_all['test']==0]
data_val=data_all[data_all['test']==1]

In [36]:
data_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,test,FamilySize,IsAlone,Title,FareBin,AgeBin,Sex_Code,Embarked_Code,Title_Code,AgeBin_Code,FareBin_Code
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0,2,0,Mr,"(-0.001, 7.896]","(16.0, 32.0]",1,2,3,1,0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,0,2,0,Mrs,"(31.275, 512.329]","(32.0, 48.0]",0,0,4,2,3
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,0,1,1,Miss,"(7.896, 14.454]","(16.0, 32.0]",0,2,2,1,1
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,0,2,0,Mrs,"(31.275, 512.329]","(32.0, 48.0]",0,2,4,2,3
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0,1,1,Mr,"(7.896, 14.454]","(32.0, 48.0]",1,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S,0,1,1,Misc,"(7.896, 14.454]","(16.0, 32.0]",1,2,1,1,1
887,1.0,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S,0,1,1,Miss,"(14.454, 31.275]","(16.0, 32.0]",0,2,2,1,2
888,0.0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,23.4500,S,0,4,0,Miss,"(14.454, 31.275]","(16.0, 32.0]",0,2,2,1,2
889,1.0,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C,0,1,1,Mr,"(14.454, 31.275]","(16.0, 32.0]",1,0,3,1,2


In [28]:
results = model_selection.cross_validate(linear_model.LogisticRegressionCV(), data_train[data_all_x_cols], data_train[Target], cv  = 5)

In [29]:
results 

{'fit_time': array([1.06938314, 0.45173764, 0.43274999, 0.48272204, 0.43075013]),
 'score_time': array([0.00299907, 0.00400639, 0.00499749, 0.00599527, 0.00299835]),
 'test_score': array([0.77653631, 0.78651685, 0.75842697, 0.79213483, 0.81460674])}

In [30]:

model=linear_model.LogisticRegressionCV()
model.fit(data_train[data_all_x_cols], data_train[Target])
predictions=model.predict(data_train[data_all_x_cols])

In [31]:
print("accuracy: ",classification_report(data_train[Target],predictions))

accuracy:                precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       549
         1.0       0.75      0.69      0.72       342

    accuracy                           0.79       891
   macro avg       0.78      0.77      0.78       891
weighted avg       0.79      0.79      0.79       891



In [32]:
name="final_model.sav"
pickle.dump(model, open(name, 'wb'))


In [33]:
#helper 

def preprocess(data_all):
    data_all['Age'].fillna(data_all['Age'].median(), inplace = True)

    #complete embarked with mode
    data_all['Embarked'].fillna(data_all['Embarked'].mode()[0], inplace = True)

    #complete missing fare with median
    data_all['Fare'].fillna(data_all['Fare'].median(), inplace = True)

    #delete the cabin feature/column and others previously stated to exclude in train dataset
    drop_column = ['PassengerId','Cabin', 'Ticket']
    data_all.drop(drop_column, axis=1, inplace = True)

    #Discrete variables
    data_all['FamilySize'] = data_all['SibSp'] + data_all['Parch'] + 1

    data_all['IsAlone'] = 1 #initialize to yes/1 is alone
    data_all['IsAlone'].loc[data_all['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    data_all['Title'] = data_all['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]


    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    data_all['FareBin'] = pd.qcut(data_all['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    data_all['AgeBin'] = pd.cut(data_all['Age'].astype(int), 5)

    #cleanup rare title names
    #print(data1['Title'].value_counts())
    stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
    title_names = (data_all['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

    #apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
    data_all['Title'] = data_all['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)

    #code categorical data
    label = LabelEncoder()
    data_all['Sex_Code'] = label.fit_transform(data_all['Sex'])
    data_all['Embarked_Code'] = label.fit_transform(data_all['Embarked'])
    data_all['Title_Code'] = label.fit_transform(data_all['Title'])
    data_all['AgeBin_Code'] = label.fit_transform(data_all['AgeBin'])
    data_all['FareBin_Code'] = label.fit_transform(data_all['FareBin'])

    return data_all


In [ ]:
# pip freeze > requirements.txt
